In [208]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [209]:
email_df = pd.read_csv("email.csv")
phish_df = pd.read_csv("Phishing_Email.csv")

In [210]:
email_df['Category'].value_counts()

Category
ham               4825
spam               747
{"mode":"full"       1
Name: count, dtype: int64

In [211]:
phish_df['Email Type'].value_counts()

Email Type
Safe Email        11322
Phishing Email     7328
Name: count, dtype: int64

In [212]:
phish_df = phish_df.iloc[:, [2, 1, 0] + list(range(3, phish_df.shape[1]))]
phish_df.columns = ['Category', 'Message'] + list(phish_df.columns[2:])
phish_df = phish_df.iloc[:, :-1]
merged_df = pd.concat([phish_df, email_df], ignore_index=True)

In [213]:
merged_df

,Category,Message
0,Safe Email,"re : 6 . 1100 , disc : uniformitarianism , re ..."
1,Safe Email,the other side of * galicismos * * galicismo *...
2,Safe Email,re : equistar deal tickets are you still avail...
3,Phishing Email,\nHello I am your hot lil horny toy.\n I am...
4,Phishing Email,software at incredibly low prices ( 86 % lower...
...,...,...
24218,ham,Will ü b going to esplanade fr home?
24219,ham,"Pity, * was in mood for that. So...any other s..."
24220,ham,The guy did some bitching but I acted like i'd...
24221,ham,Rofl. Its true to its name


In [214]:
merged_df.Category.unique()

array(['Safe Email', 'Phishing Email', 'ham', 'spam', '{"mode":"full"'],
      dtype=object)

In [215]:
valid_labels = ['Safe Email', 'Phishing Email', 'ham', 'spam']
filtered_df = merged_df[merged_df['Category'].isin(valid_labels)].copy()
label_mapping = {'ham': 0, 'spam': 1,'Safe Email': 0, 'Phishing Email': 1}
filtered_df['Category'] = filtered_df['Category'].map(label_mapping)

In [216]:
filtered_df

,Category,Message
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ..."
1,0,the other side of * galicismos * * galicismo *...
2,0,re : equistar deal tickets are you still avail...
3,1,\nHello I am your hot lil horny toy.\n I am...
4,1,software at incredibly low prices ( 86 % lower...
...,...,...
24217,1,This is the 2nd time we have tried 2 contact u...
24218,0,Will ü b going to esplanade fr home?
24219,0,"Pity, * was in mood for that. So...any other s..."
24220,0,The guy did some bitching but I acted like i'd...


In [217]:
label_counts = filtered_df['Category'].value_counts()

count_0 = label_counts.get(0, 0)  
count_1 = label_counts.get(1, 0)  

total = count_0 + count_1
proportion_0 = count_0 / total if total > 0 else 0
proportion_1 = count_1 / total if total > 0 else 0

print(f"Count of 0 (Safe/ham): {count_0}")
print(f"Count of 1 (Phishing/spam): {count_1}")
print(f"Proportion of 0: {proportion_0:.2%}")
print(f"Proportion of 1: {proportion_1:.2%}")

Count of 0 (Safe/ham): 16147
Count of 1 (Phishing/spam): 8075
Proportion of 0: 66.66%
Proportion of 1: 33.34%


In [218]:
ham = filtered_df[filtered_df['Category'] == 0]
spam = filtered_df[filtered_df['Category'] == 1]
ham_downsampled = ham.sample(n=len(spam), random_state=42)
balanced_df = pd.concat([ham_downsampled, spam], axis=0).reset_index(drop=True)
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(balanced_df['Category'].value_counts())

Category
0    8075
1    8075
Name: count, dtype: int64


In [219]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zlp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zlp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zlp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\zlp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [220]:
stop_words = stopwords.words('english')
print("Number of stopwords:", len(stop_words))
print("Stopwords list:", stop_words)

Number of stopwords: 179
Stopwords list: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not

In [221]:
stop_words = set(stopwords.words('english')) - {'all', 'any', 'each', 'few', 'more', 'most', 'only', 'too', 'very'}

In [222]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation.replace('!', '').replace('$', '').replace('€', '').replace('¥', '')))

    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() or word in {'!', '$', '€', '¥'}]
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

balanced_df['Processed_Message'] = balanced_df['Message'].apply(preprocess_text)

In [223]:
balanced_df

,Category,Message,Processed_Message
0,0,"Folks,\n \nmy first time posting - have a bit ...",folk first time posting bit unix experience ne...
1,1,download faster hows it been going ? make your...,download faster hows going make ordinary k mod...
2,0,dialectal variation harold orton centenary con...,dialectal variation harold orton centenary con...
3,1,re : here is our opportunity carey dusenburyco...,opportunity carey dusenburycongeal banish taxi...
4,0,sum : representing retroflex summary : represe...,sum representing retroflex summary representin...
...,...,...,...
16145,1,re : cereal i did not talk with pete for long ...,cereal talk pete long got email yesterday said...
16146,0,calpine force majeure attached is a summary of...,calpine force majeure attached summary day cal...
16147,0,baltics 1998 baltics 1998 - - - first announce...,baltic baltic first announcement intensive lan...
16148,1,Adult 18 Content Your video will be with you s...,adult content video shortly


In [224]:
keyword_dict = {
    "lure": {
        "low": ["free", "offer", "deal", "discount", "gift", "limited", "exclusive", "$", "€", "¥"], 
        "medium": ["prize", "exclusive", "romantic", "sex", "dating", "date", "opportunity"],
        "high": ["jackpot", "win big", "life-changing", "perfect match", "irresistible", "secret", "unlimited"]
    },
    "urgency": {
        "low": ["soon", "act", "hurry", "quick", "soon", "today"], 
        "medium": ["urgent", "limited", "exclusive", "last chance", "don’t miss", "only available", "flash sale", "this week"],
        "high": ["!", "now", "immediately", "hurry up", "act fast", "don’t wait", "time’s running out", "offer ends", "limited stock", "rush", "grab now", "quickly", "final call"],
    },
}

In [225]:
balanced_df['lure_score'] = 0
balanced_df['urgency_score'] = 0

def calculate_scores(text):
    lure_score = 0
    urgency_score = 0

    for level, words in keyword_dict['lure'].items():
        for word in words:
            if word in text:
                if level == "low":
                    lure_score += 1
                elif level == "medium":
                    lure_score += 2
                elif level == "high":
                    lure_score += 3

    for level, words in keyword_dict['urgency'].items():
        for word in words:
            if word in text:
                if level == "low":
                    urgency_score += 1
                elif level == "medium":
                    urgency_score += 2
                elif level == "high":
                    urgency_score += 3

    return lure_score, urgency_score

balanced_df[['lure_score', 'urgency_score']] = balanced_df['Processed_Message'].apply(lambda text: pd.Series(calculate_scores(text)))

In [226]:
n_bins = 3
balanced_df['lure_level'] = pd.cut(balanced_df['lure_score'], bins=n_bins, labels=["low", "medium", "high"])
balanced_df['urgency_level'] = pd.cut(balanced_df['urgency_score'], bins=n_bins, labels=["low", "medium", "high"])

In [227]:
balanced_df.head()

,Category,Message,Processed_Message,lure_score,urgency_score,lure_level,urgency_level
0,0,"Folks,\n \nmy first time posting - have a bit ...",folk first time posting bit unix experience ne...,0,1,low,low
1,1,download faster hows it been going ? make your...,download faster hows going make ordinary k mod...,0,3,low,low
2,0,dialectal variation harold orton centenary con...,dialectal variation harold orton centenary con...,9,11,medium,medium
3,1,re : here is our opportunity carey dusenburyco...,opportunity carey dusenburycongeal banish taxi...,2,0,low,low
4,0,sum : representing retroflex summary : represe...,sum representing retroflex summary representin...,0,1,low,low


In [228]:
balanced_df['lure_level'] = balanced_df['lure_level'].map({"low": 0, "medium": 1, "high": 2})
balanced_df['urgency_level'] = balanced_df['urgency_level'].map({"low": 0, "medium": 1, "high": 2})

In [229]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(balanced_df[['lure_level', 'urgency_level']])
X_additional = np.hstack([balanced_df[['lure_level', 'urgency_level']].values, encoded_features])

In [231]:
X_additional = X_additional.astype(np.float64)

In [236]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import numpy as np
from scipy.stats import chi2_contingency
from scipy.sparse import hstack, csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [238]:
def evaluate_model(y_test, y_pred):
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")

In [237]:
n_train = int(len(balanced_df) * 0.7)

vectorizer = TfidfVectorizer()  
X_tfidf = vectorizer.fit_transform(balanced_df['Processed_Message'])
X_tfidf_train = X_tfidf[:n_train]  
X_tfidf_test = X_tfidf[n_train:]

X_combined = hstack([X_tfidf, X_additional]) 
X_combined = csr_matrix(X_combined)
X_combined_train = X_combined[:n_train]
X_combined_test = X_combined[n_train:]

y = balanced_df['Category']
y_train = y.iloc[:n_train].values  
y_test = y.iloc[n_train:].values

In [242]:
# NB only tfidf
model_tfidf = MultinomialNB()
model_tfidf.fit(X_tfidf_train, y_train)
y_pred = model_tfidf.predict(X_tfidf_test)
evaluate_model(y_test, y_pred)

Precision: 0.91
Recall: 0.97
F1 Score: 0.94
Accuracy: 0.94
False Positives (FP): 225
False Negatives (FN): 74


In [243]:
# NB combine
model_combine = MultinomialNB()
model_combine.fit(X_combined_train, y_train)
y_pred = model_combine.predict(X_combined_test)
evaluate_model(y_test, y_pred)

Precision: 0.92
Recall: 0.95
F1 Score: 0.94
Accuracy: 0.94
False Positives (FP): 195
False Negatives (FN): 111


In [244]:
# RF only tfidf
model_tfidf = RandomForestClassifier(random_state=42)
model_tfidf.fit(X_tfidf_train, y_train)
y_pred = model_tfidf.predict(X_tfidf_test)
evaluate_model(y_test, y_pred)

Precision: 0.94
Recall: 0.96
F1 Score: 0.95
Accuracy: 0.95
False Positives (FP): 144
False Negatives (FN): 99


In [245]:
# RF combine
model_combine = RandomForestClassifier(random_state=42)
model_combine.fit(X_combined_train, y_train)
y_pred = model_combine.predict(X_combined_test)
evaluate_model(y_test, y_pred)

Precision: 0.94
Recall: 0.96
F1 Score: 0.95
Accuracy: 0.95
False Positives (FP): 148
False Negatives (FN): 99


In [246]:
# MLP only tfidf
model_tfidf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
model_tfidf.fit(X_tfidf_train, y_train)
y_pred = model_tfidf.predict(X_tfidf_test)
evaluate_model(y_test, y_pred)

Precision: 0.93
Recall: 0.96
F1 Score: 0.95
Accuracy: 0.95
False Positives (FP): 175
False Negatives (FN): 86


In [247]:
# MLP combine
model_combine = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
model_combine.fit(X_combined_train, y_train)
y_pred = model_combine.predict(X_combined_test)
evaluate_model(y_test, y_pred)

Precision: 0.92
Recall: 0.97
F1 Score: 0.95
Accuracy: 0.94
False Positives (FP): 197
False Negatives (FN): 75
